<a href="https://colab.research.google.com/github/Greeshu-09/AISECT-AIML/blob/main/Cross_validation_KNN_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#importing the Libraies
import numpy as np
import pandas as pd

In [8]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [9]:
#Checking the first 5 values
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [10]:
#checking the last 5 values
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [11]:
#Encoding the values using onehot coding
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [12]:
#Assigning the independent and dependent value
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [13]:
#standardising the values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [14]:
#importing the algorithm
from sklearn.neighbors import KNeighborsClassifier

In [15]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
# Small parameter grid for faster execution
param_grid = {
    'n_neighbors': [3, 5, 7],          # small number of neighbors
    'weights': ['uniform'],             # faster than 'distance'
    'metric': ['euclidean', 'manhattan']  # simple distance metrics
}

# GridSearchCV setup
grid = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,          # 3-fold CV for speed
    refit=True,
    verbose=2,
    n_jobs=-1      # parallelize if possible
)

# Fit the model
grid.fit(X, y)

# Best parameters & score
print("Best Parameters:", grid.best_params_)
print("Best f1_weighted Score:", grid.best_score_)

# Best estimator for predictions
best_knn = grid.best_estimator_

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best Parameters: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
Best f1_weighted Score: 0.9005544953529486


In [16]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=3)

In [17]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')


In [18]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [19]:
# Save Best Model
import pickle
filename = "KNN Classifier.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [23]:
  #reading the saved file
pickle.load(open("/content/KNN Classifier.sav","rb"))

KNeighborsClassifier(metric='euclidean', n_neighbors=7)

In [21]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003136,0.000843,0.007405,0.000811,euclidean,3,uniform,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",0.917716,0.908219,0.778108,0.868014,0.063691,6
1,0.001957,0.000059,0.006691,0.000722,euclidean,5,uniform,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei...",0.932986,0.916138,0.837819,0.895648,0.041466,3
2,0.001846,0.000043,0.006741,0.000644,euclidean,7,uniform,"{'metric': 'euclidean', 'n_neighbors': 7, 'wei...",0.917716,0.923996,0.859952,0.900554,0.028824,1
3,0.001912,0.000064,0.006582,0.000814,manhattan,3,uniform,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei...",0.924599,0.923516,0.815244,0.887786,0.051297,4
4,0.001914,0.000040,0.007032,0.000656,manhattan,5,uniform,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei...",0.917290,0.916138,0.822959,0.885462,0.044199,5


In [22]:
# Get user input
age = int(input("Enter Age: "))
salary = float(input("Enter Estimated Salary: "))
gender_male = int(input("Enter Gender (0 for Female, 1 for Male): "))

# Preprocess input
# The order of features should match the training data: 'Age', 'EstimatedSalary', 'Gender_Male'
user_input = np.array([[age, salary, gender_male]])
user_input_scaled = sc.transform(user_input)

# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))

# Make prediction
prediction = loaded_model.predict(user_input_scaled)

# Display the prediction
if prediction[0] == 1:
    print("Likely to purchase.")
else:
    print("Unlikely to purchase.")

Enter Age: 19
Enter Estimated Salary: 19000
Enter Gender (0 for Female, 1 for Male): 1
Unlikely to purchase.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
